In [1]:
import numpy as np
from pathlib import Path
import pandas as pd

# load admission, patients, labevents, chartevents
admissions = pd.read_csv("../data/ADMISSIONS.csv")
patients = pd.read_csv("../data/PATIENTS.csv")

# demographics: age, gender
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
patients['DOB']         = pd.to_datetime(patients['DOB'])

df = (
    admissions[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME']]
    .merge(patients[['SUBJECT_ID', 'DOB', 'GENDER']], on='SUBJECT_ID', how='left')
)

# calculate age
df['AGE'] = df['ADMITTIME'].dt.year - df['DOB'].dt.year

mask = (
    (df['ADMITTIME'].dt.month  < df['DOB'].dt.month) |
    ((df['ADMITTIME'].dt.month == df['DOB'].dt.month) &
     (df['ADMITTIME'].dt.day   < df['DOB'].dt.day))
)
df.loc[mask, 'AGE'] -= 1

# cap at 90 per MIMIC policy
df.loc[df['AGE'] > 89, 'AGE'] = 90

age = df.set_index('SUBJECT_ID')['AGE']
gender = df.set_index('SUBJECT_ID')['GENDER']
# print(gender.head())

# load chart values
chart_values = pd.read_csv("../data/CHARTEVENTS.csv",
                           usecols=['HADM_ID', 'ITEMID', 'VALUENUM'],
                           low_memory=True)
# define needed features
chart_itemids = {
    211:     "HeartRate",
    618:     "RespRate", # respiratory rate
    220050:  "Art_SBP", # arterial blood pressure systolic
    220179:  "NIBP_SBP", # non invasive blood pressure systolic
    220051:  "Art_DBP", # diastolic
    220180:  "NIBP_DBP",
    220052:  "Art_MBP", # mean
    220181:  "NIBP_MBP",
    223761:  "Temp_F",
    228232:  "SpO2" # oxygen saturation
}
# filter
mask = chart_values["ITEMID"].isin(chart_itemids.keys())
chart_filtered = chart_values.loc[mask].copy()
chart_filtered["measurement"] = chart_filtered["ITEMID"].map(chart_itemids)
chart_agg = (
    chart_filtered
    .groupby(["HADM_ID", "measurement"])["VALUENUM"]
    .mean()
    .unstack(fill_value=np.nan)
    .reset_index()
)

print(chart_agg.head())


measurement  HADM_ID    Art_DBP    Art_MBP     Art_SBP   HeartRate   NIBP_DBP  \
0             100001        NaN        NaN         NaN         NaN  92.780822   
1             100003        NaN        NaN         NaN         NaN  60.615385   
2             100006        NaN        NaN         NaN  108.630252        NaN   
3             100007        NaN        NaN         NaN   79.444444        NaN   
4             100009  53.666667  71.095238  116.761905         NaN  49.722222   

measurement    NIBP_MBP    NIBP_SBP   RespRate  SpO2     Temp_F  
0            111.852941  174.191781        NaN   NaN  98.929167  
1             73.980769  116.519231        NaN   NaN  97.245455  
2                   NaN         NaN  18.647059   NaN        NaN  
3                   NaN         NaN  19.185185   NaN        NaN  
4             63.228571  106.916667        NaN   NaN  98.414286  


In [2]:
# load lab values
lab_values = pd.read_csv("../data/LABEVENTS.csv",
                         usecols=['HADM_ID', 'ITEMID', 'VALUENUM'],
                         low_memory=True)
lab_itemids = {
    # in blood fluid
    50809: "Glucose",
    50912: "Creatinine",
    50983: "Sodium",
    50971: "Potassium",
    50882: "Bicarbonate",
    50902: "Chloride",
    51006: "Urea_Nitrogen",
    51301: "WBC", # white blood cells
    51221: "Hematocrit",
    51222: "Hemoglobin",
    51265: "Platelet_Count",
    51049: "Total_Bilirubin",
    50862: "Albumin",
    50813: "Lactate",
    50820: "pH",
    50818: "pCO2",
    50821: "pO2"
}
relevant_itemids = list(lab_itemids.keys())
lab_filtered = lab_values[lab_values["ITEMID"].isin(relevant_itemids)].copy()
lab_filtered["measurement"] = lab_filtered["ITEMID"].map(lab_itemids)
lab_agg = (
    lab_filtered
    .groupby(["HADM_ID", "measurement"])["VALUENUM"]
    .mean()
    .unstack(fill_value=np.nan)
    .reset_index()
)
lab_agg.columns.name = None
print(lab_agg.head())


    HADM_ID  Albumin  Bicarbonate    Chloride  Creatinine     Glucose  \
0  100001.0      NaN    19.714286  107.214286    2.214286         NaN   
1  100003.0      2.4    19.666667  105.833333    1.083333         NaN   
2  100006.0      2.0    29.384615   97.615385    0.638462         NaN   
3  100007.0      NaN    24.888889  106.222222    0.666667         NaN   
4  100009.0      4.3    25.200000  103.857143    0.783333  132.833333   

   Hematocrit  Hemoglobin  Lactate  Platelet_Count  Potassium      Sodium  \
0   32.157143   11.114286      1.9      364.428571   4.035714  139.571429   
1   27.781818    9.214286      1.1      145.285714   4.783333  132.000000   
2   31.853846   10.600000      4.5      228.615385   4.076923  131.384615   
3   30.600000   10.187500      2.5      199.750000   3.955556  139.111111   
4   35.914286   12.557143      1.3      142.571429   4.242857  137.714286   

   Total_Bilirubin  Urea_Nitrogen        WBC  pCO2        pH    pO2  
0              NaN      28.5

In [3]:
# CPT codes
# chartdate for timing of procedure
cpt_events = pd.read_csv('../data/CPTEVENTS.csv',
                         usecols=['SUBJECT_ID','HADM_ID','CHARTDATE','CPT_NUMBER'])
d_cpt      = pd.read_csv('../data/D_CPT.csv',
                         usecols=['SECTIONHEADER','SUBSECTIONHEADER',
                                  'MINCODEINSUBSECTION','MAXCODEINSUBSECTION'])

# map code to section
intervals = pd.IntervalIndex.from_arrays(
    left=d_cpt['MINCODEINSUBSECTION'],
    right=d_cpt['MAXCODEINSUBSECTION'],
    closed='both'
)
d_cpt = d_cpt.assign(interval=intervals)

# cpt timing
cpt_events['CHARTDATE'] = pd.to_datetime(cpt_events['CHARTDATE'])
cpt_events = cpt_events.merge(admissions, on='HADM_ID', how='left')
cpt_events['days_since_admission'] = (
    cpt_events['CHARTDATE'] - cpt_events['ADMITTIME']
).dt.days

# look up
def lookup_cpt_sections(cpt_num):
    mask = intervals.contains(cpt_num)
    matches = d_cpt[mask]
    if matches.empty:
        return pd.Series({'sections': None, 'subsections': None})
    else:
        return pd.Series({
            'sections':    ';'.join(matches['SECTIONHEADER'].astype(str)),
            'subsections': ';'.join(matches['SUBSECTIONHEADER'].astype(str)),
        })

cpt_events[['section','subsection']] = (
    cpt_events['CPT_NUMBER']
              .apply(lookup_cpt_sections)
)
agg = cpt_events.groupby('HADM_ID').agg(
    total_cpt_count    = ('CPT_NUMBER','size'),
    unique_cpt_codes   = ('CPT_NUMBER','nunique'),
    distinct_sections  = ('section','nunique'),
    distinct_subsects  = ('subsection','nunique'),
)

timing_agg = cpt_events.groupby('HADM_ID').agg(
first_cpt_day   = ('days_since_admission','min'),   # e.g. 0 = same day
    last_cpt_day    = ('days_since_admission','max'),
    cpt_span_days   = ('days_since_admission', lambda x: x.max() - x.min()),
)

code_agg = cpt_events.groupby('HADM_ID').agg(
    min_cpt_number     = ('CPT_NUMBER','min'),
    max_cpt_number     = ('CPT_NUMBER','max')
    # cpt_number_range   = ('cpt_number', lambda x: x.max() - x.min()),
)

# high risk indicator
# high_risk_secs = {'Surgery','Cardiovascular','Neurosurgery'}  # customize as needed
# cpt_events['is_high_risk_section'] = cpt_events['section'].isin(high_risk_secs).astype(int)
#
# high_risk_agg = cpt_events.groupby('hadm_id').agg(
#     had_any_high_risk = ('is_high_risk_section','max'),
#     high_risk_count   = ('is_high_risk_section','sum')
# )
cpt_features = (
    agg
    # .join(high_risk_agg,  how='left')
    .join(timing_agg,     how='left')
    .join(code_agg,       how='left')
    .fillna(0)
    .reset_index()
)
print(cpt_features.head())

/var/folders/0q/ly2yy0zn3jx47gypxrpcd5vm0000gn/T/ipykernel_84566/4225125244.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  cpt_events = pd.read_csv('../data/CPTEVENTS.csv',


   HADM_ID  total_cpt_count  unique_cpt_codes  distinct_sections  \
0   100001                8                 4                  1   
1   100003                8                 6                  3   
2   100006               10                 4                  1   
3   100007                5                 3                  1   
4   100009               11                 8                  3   

   distinct_subsects  first_cpt_day  last_cpt_day  cpt_span_days  \
0                  2            0.0           0.0            0.0   
1                  4            0.0           0.0            0.0   
2                  1            0.0           0.0            0.0   
3                  2            0.0           0.0            0.0   
4                  4            1.0           1.0            0.0   

   min_cpt_number  max_cpt_number  
0         99232.0         99291.0  
1         36556.0         99291.0  
2         99231.0         99238.0  
3         99231.0         99254.0  
4 

In [4]:
# ICD 9 code
diag_agg = pd.read_csv("diag_features.csv")
proc_agg = pd.read_csv("proc_features.csv")

# ICU stay
icu_agg = pd.read_csv("icu_features.csv")
# prepare the feature data frame
demo_feats = (
    df[['HADM_ID','SUBJECT_ID', 'AGE', 'GENDER']]
    .drop_duplicates(subset='HADM_ID')
    .set_index('HADM_ID')
)
lab_agg = lab_agg.dropna(subset=['HADM_ID']).set_index('HADM_ID')
chart_agg = chart_agg.dropna(subset=['HADM_ID']).set_index('HADM_ID')
cpt_features = cpt_features.dropna(subset=['HADM_ID']).set_index('HADM_ID')
diag_agg = diag_agg.dropna(subset=['HADM_ID']).set_index('HADM_ID')
proc_agg = proc_agg.dropna(subset=['HADM_ID']).set_index('HADM_ID')
icu_agg = icu_agg.dropna(subset=['HADM_ID']).set_index('HADM_ID')
feature_df = (
    demo_feats
      .join(lab_agg, how='left')
      .join(chart_agg, how='left')
      .join(cpt_features, how='left')
      .join(diag_agg, how='left')
      .join(proc_agg, how='left')
      .join(icu_agg, how='left')
      .reset_index()
)

print(feature_df.shape)
print(feature_df.columns)
print(feature_df.head())

(58976, 95)
Index(['HADM_ID', 'SUBJECT_ID', 'AGE', 'GENDER', 'Albumin', 'Bicarbonate',
       'Chloride', 'Creatinine', 'Glucose', 'Hematocrit', 'Hemoglobin',
       'Lactate', 'Platelet_Count', 'Potassium', 'Sodium', 'Total_Bilirubin',
       'Urea_Nitrogen', 'WBC', 'pCO2', 'pH', 'pO2', 'Art_DBP', 'Art_MBP',
       'Art_SBP', 'HeartRate', 'NIBP_DBP', 'NIBP_MBP', 'NIBP_SBP', 'RespRate',
       'SpO2', 'Temp_F', 'total_cpt_count', 'unique_cpt_codes',
       'distinct_sections', 'distinct_subsects', 'first_cpt_day',
       'last_cpt_day', 'cpt_span_days', 'min_cpt_number', 'max_cpt_number',
       'total_diag_count', 'unique_diag_codes', 'distinct_diag_chapters',
       'diag_chap_count_0', 'diag_chap_count_1', 'diag_chap_count_2',
       'diag_chap_count_3', 'diag_chap_count_4', 'diag_chap_count_5',
       'diag_chap_count_6', 'diag_chap_count_7', 'diag_chap_count_8',
       'diag_chap_count_9', 'diag_chap_count_10', 'diag_chap_count_11',
       'diag_chap_count_12', 'diag_chap_count_13

In [5]:
# target dataframe
adm_counts = (
    admissions
      .groupby("SUBJECT_ID")["HADM_ID"]
      .nunique()
      .reset_index(name="n_admissions")
)

# 2. Filter to patients with >1 admissions & drop outlier
p99 = adm_counts["n_admissions"].quantile(0.99)
multi_adm = adm_counts[
    (adm_counts["n_admissions"] > 1) &
    (adm_counts["n_admissions"] <= p99)
].copy()

print(f"99th percentile cutoff = {p99:.0f} admissions")
print(multi_adm)

admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])
# sort with order
admissions = admissions.sort_values(['SUBJECT_ID', 'ADMITTIME'])
admissions['next_admittime'] = (
    admissions
      .groupby('SUBJECT_ID')['ADMITTIME']
      .shift(-1)
)
admissions['readmit_gap'] = (
    admissions['next_admittime'] - admissions['DISCHTIME']
).dt.days
admissions['readmission_30'] = (
    admissions['readmit_gap']
      .between(1, 30)
      .fillna(0)
      .astype(int)
)
admissions['readmission_60'] = (
    admissions['readmit_gap']
      .between(1, 60)        # 1–60 days
      .astype(int)
)
admissions['n_admissions'] = (
    admissions.groupby('SUBJECT_ID')['HADM_ID']
              .transform('nunique')
)
mask = (
    (admissions['n_admissions'] == 1) |
    (admissions['next_admittime'].notna())
)
target_df = admissions.loc[mask,
    ['SUBJECT_ID','HADM_ID','readmission_30','readmission_60']
].copy()
print(target_df.sample(10))

99th percentile cutoff = 5 admissions
       SUBJECT_ID  n_admissions
13             17             2
17             21             2
19             23             2
29             34             2
31             36             3
...           ...           ...
46470       99822             3
46488       99883             2
46491       99897             2
46498       99923             2
46513       99982             3

[7256 rows x 2 columns]
       SUBJECT_ID  HADM_ID  readmission_30  readmission_60
8763         7462   110631               0               0
48493       67408   118358               0               0
26628       24717   152186               0               0
49834       74856   193325               0               0
6555         5816   181634               0               0
51014       77099   194038               0               0
16080       12973   190689               0               0
17455       16747   161957               0               0
57933       99298   12

In [6]:
# prepare the final data frame
final_df = feature_df.merge(
    target_df[['HADM_ID','readmission_30','readmission_60']],
    on='HADM_ID',
    how='inner'
)
print(final_df.shape)
print(final_df[['HADM_ID','readmission_30','readmission_60']].head())
final_df.to_csv("../src/dataframe_clean.csv", index=False)

(51439, 97)
    HADM_ID  readmission_30  readmission_60
0  165315.0               0               0
1  152223.0               0               0
2  161859.0               0               0
3  129635.0               0               0
4  197661.0               0               0
